In [12]:
# 3.0 LLMs and Chat Models
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-instruct")
chat = ChatOpenAI()

a = llm.predict("How many planets are there?")
b = chat.predict("How many planets are there?")

a, b

('\n\nAs of October 2021, there are eight planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. Pluto was previously considered a planet but is now classified as a dwarf planet.',
 'There are eight planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.')

In [8]:
# 3.1 Predict Messages
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
  SystemMessage(content="You are a geography expert. And you only reply in Italian.",),
  AIMessage(content="Ciao, mi chiamo Paolo!"),
  HumanMessage(content="What is the distance between Mexico and Thailand. Also, what is your name?")
]

chat.predict_messages(messages)

AIMessage(content='La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri. Come posso aiutarti oggi?')

In [13]:
# 3.2 Prompt Templates
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

chat = ChatOpenAI(
  temperature=0.1
)

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}")
prompt = template.format(country_a="Mexico", country_b="Thailand")

chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 9,500 miles (15,300 kilometers) when measured in a straight line.'

In [11]:
# 3.2 Prompt Templates
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
  temperature=0.1
)

template = ChatPromptTemplate.from_messages([
  ("system", "You are a geography expert. And you only reply in {language}."),
  ("ai", "Ciao, mi chiamo {name}!"),
  ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?")
])

prompt = template.format_messages(
  language="Greek",
  name="Socrates",
  country_a="Mexico",
  country_b="Thailand",
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα. Το όνομά μου είναι Σωκράτης. Πώς μπορώ να βοηθήσω;')

In [18]:
# 3.3 OutputParser and LCEL (LangChain Expression Language)
from langchain.chat_models import ChatOpenAI
from langchain.schema import BaseOutputParser

chat = ChatOpenAI(
  temperature=0.1
)

class CommaOutputParser(BaseOutputParser):
   
   def parse(self, text):
      item = text.strip().split(",")    # 텍스트의 앞뒤 공백 제거 후 comma(,)로 잘라서 array로 반환 
      return list(map(str.strip, item)) # 각 item에 strip 함수 적용
   
template = ChatPromptTemplate.from_messages([
  ("system", "You are a list generating machine. Evenything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else."),
  ("human", "{question}"),
])

prompt = template.format_messages(
   max_items=10, 
   question="What are the colors?"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)


['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'black',
 'white',
 'pink',
 'brown']

In [21]:
# 3.3 OutputParser and LCEL (LangChain Expression Language)
from langchain.chat_models import ChatOpenAI
from langchain.schema import BaseOutputParser

chat = ChatOpenAI(
  temperature=0.1
)

class CommaOutputParser(BaseOutputParser):
   
   def parse(self, text):
      item = text.strip().split(",")    # 텍스트의 앞뒤 공백 제거 후 comma(,)로 잘라서 array로 반환 
      return list(map(str.strip, item)) # 각 item에 strip 함수 적용
   
template = ChatPromptTemplate.from_messages([
  ("system", "You are a list generating machine. Evenything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else."),
  ("human", "{question}"),
])

chain = template | chat | CommaOutputParser()

chain.invoke({
   "max_items": 5,
   "question": "What are the pokemons?"
})

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']

In [5]:
# 3.4 Chaining Chains
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
  temperature=0.1,
  streaming=True,
  callbacks=[StreamingStdOutCallbackHandler()]
)

chef_prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
  ("human", "I want to cook {cuisine} food.")
])

chef_chain = chef_prompt | chat

veg_chef_prompt = ChatPromptTemplate.from_messages([
  ("system", '''You are a vegetarian chef specialized on making traditional recipies vegetarian. 
   You find alternative ingredients and explain their preparation. 
   You don't radically modify the recipe. 
   If there is no alternative for a food just say you don't know how to replace it.'''),
  ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({
  "cuisine": "indian"
})

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's start with a classic and popular dish - Chicken Tikka Masala. Here's a simple recipe for you to try at home:

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 tablespoons vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1 tablespoon ginger, minced
- 1 can (14 oz) tomato sauce
- 1 tablespoon garam masala
- 1 teaspoon ground cumin
- 1 teaspoon ground coriander
- 1 teaspoon paprika
- 1/2 teaspoon turmeric
- Salt and pepper to taste
- Fresh cilantro, chopped for garnish
- Cooked rice or naan bread for serving

Instructions:
1. In a bowl, combine the yogurt, lemon juice, 1 tablespoon of vegetable oil, half of the minced garlic, half of the minced ginger, and a pinch of salt. Add the chicken pieces and mix well to coat. Cover and marinate in the refrigerator for at least 1 hour, or overnight for best re

AIMessageChunk(content="For a vegetarian version of Chicken Tikka Masala, you can replace the chicken with a plant-based alternative such as tofu or paneer. Here's how you can prepare these alternatives:\n\n1. **Tofu**: \n   - Use firm or extra-firm tofu for this recipe.\n   - Drain the tofu and press it to remove excess water. You can do this by wrapping the tofu block in a clean kitchen towel or paper towels and placing a heavy object on top for about 15-30 minutes.\n   - Cut the tofu into bite-sized cubes and follow the same marinating process as you would with the chicken. Tofu absorbs flavors well, so the marination time can be shorter than with meat.\n   - Instead of baking, you can pan-fry the marinated tofu in a bit of oil until it's golden brown and slightly crispy on the outside.\n\n2. **Paneer**:\n   - Paneer is a type of Indian cheese that holds its shape well when cooked.\n   - Cut the paneer into bite-sized cubes and add them to the sauce towards the end of cooking to pre